# SBTi Temperature Alignment tool examples
This notebook provides some examples on how to use the SBTi Temperature Alignment tool.

> This notebook is meant to run on Google Colab. If you're running the notebook locally, use the normal "1_getting_started.ipynb" notebook

## Install the SBTi Python module

In [ ]:
!pip install git+git://github.com/OFBDABV/SBTi

In [2]:
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario
from SBTi.target_validation import TargetValidation
import pandas as pd

## Download the dummy data provider

In [1]:
import urllib.request
import os
os.mkdir("data")
urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data'

#### Logging
The SBTi module uses the Python standard library logging utilities to send log messages. The log level can be changed according to the user's needs.

In [3]:
import logging
root_logger = logging.getLogger()
root_logger.setLevel("INFO")

### Create a data provider
Data providers let you connect to the data source of your choice.

In [4]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")

### Load your portfolio
In our case the portfolio is stored as a CSV file. The portfolio should at least have an "id" (the identifier of the company) and a "proportion" (the weight of the company in your portfolio e.g. the value of the shares you hold) column.

In [5]:
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")

In [6]:
portfolio.head(5)

,company_name,company_id,ISIN,weights,investment_value,engagement_target
0,"Advanced Micro Devices, Inc",US0079031078,US0079031078,6.36%,35000000,1.0
1,Adobe Systems Inc.,US00724F1012,US00724F1012,1.82%,10000000,NaN
2,Capgemini Group,FR0000125338,FR0000125338,1.82%,10000000,NaN
3,"Cisco Systems, Inc.",US17275R1023,US17275R1023,1.82%,10000000,NaN
4,Coca-Cola HBC AG,CH0198251305,CH0198251305,1.82%,10000000,NaN


To load the data from the data provider, we have to pass a list of dictionaries, containing a "company_name" and "company_id".

In [7]:
companies = portfolio[["company_id", "company_name"]].to_dict(orient="records")

### Load the data
We'll call the data provider and ask it to provide us with the targets and the company data.

In [8]:
company_data = SBTi.data.get_company_data([provider], companies)
target_data = SBTi.data.get_targets([provider], companies)
company_data = pd.merge(left=company_data,
                        right=portfolio[
                            [column
                                for column in portfolio.columns
                                if column not in ["company_name"]]],
                        how="left",
                        on=["company_id"])

### Validate the targets
We've now got our portfolio, amended with target data. Now we need to make sure that all the targets are valid (e.g. they haven't been reached yet and their end date isn't in the past).

In [9]:
target_validation = TargetValidation(target_data, company_data)
portfolio_data = target_validation.target_validation()

### Calculate the temperature scores
In the amended portfolio you'll find your original portfolio, amended with the emissions and the temperature score.

In [10]:
temperature_score = TemperatureScore()
amended_portfolio = temperature_score.calculate(portfolio_data)

In [11]:
amended_portfolio.head(10)

,company_name,company_id,ISIC,Country,Region,Industry_lvl1,Industry_lvl2,Industry_lvl3,Industry_lvl4,sector,...,annual_reduction_rate,slope,samplesize,model,variable,param,intercept,r2,temperature_score,temperature_results
0,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,0.060000,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,1.298150,0
1,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,0.138571,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,-0.373002,0
2,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,NaN,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,0.082679,1
3,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,0.027785,slope15,128,4,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,1.829468,0
4,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,NaN,slope15,128,4,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,3.200000,1
5,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,NaN,slope15,128,4,Emissions|Kyoto Gases,-0.312329,2.697261,0.829523,2.826291,1
6,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,NaN,slope30,128,4,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.200000,1
7,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,NaN,slope30,128,4,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.200000,1
8,"Advanced Micro Devices, Inc",US0079031078,NaN,United States,North America,NaN,NaN,NaN,NaN,Industrials,...,NaN,slope30,128,4,Emissions|Kyoto Gases,-0.483331,2.841338,0.925768,3.200000,1
9,Adobe Systems Inc.,US00724F1012,NaN,United States,North America,NaN,NaN,NaN,NaN,Utilities,...,NaN,slope5,128,4,Emissions|Kyoto Gases,-0.212692,2.574303,0.609039,3.200000,1


### Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb"). The termperature scores are calculated per time-frame/scope combination.

In [12]:
def print_aggregations(aggregations):
    for time_frame, time_frame_values in aggregations.items():
        for scope, scope_values in time_frame_values.items():
            print("{} - {}: {} degrees celcius".format(time_frame, scope, scope_values["all"]["score"]))

### Portfolio coverage

Gives insights in the coverage of portfolio entities that has set GHG emissions reduction targets, STBi-validated or otherwise. Companies with SBTi status Approved have a score of 100%, all others have a score of 0%. 

This can be done use different aggregation methods. Here we'll use the "Weighted Average Temperature Score". For more details, have a look at notebook 3 ("3_portfolio_aggregations.ipynb").

In [13]:
portfolio_coverage_tvp = PortfolioCoverageTVP()
coverage = portfolio_coverage_tvp.get_portfolio_coverage(amended_portfolio.copy(), PortfolioAggregationMethod.WATS)
round(coverage * 100) / 100

35.45